1. Create a keyspace with a name that matches your dataset.

2. Import your files into Cassandra.

3. Check that the data import went well by consulting the numbers of rows in the tables

CREATE TABLE dbs.netflix_userbase (
    user_id int PRIMARY KEY,
    age int,
    country text,
    device text,
    gender text,
    join_date text,
    last_payment_date text,
    monthly_revenue int,
    plan_duration text,
    subscription_type text
) WITH additional_write_policy = '99PERCENTILE'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.UnifiedCompactionStrategy'}
    AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99PERCENTILE';
    
CREATE TABLE dbs.customers (
    customerid int PRIMARY KEY,
    age int,
    annual_income int,
    family_size int,
    gender text,
    profession text,
    spending_score int,
    work_experience int
) WITH additional_write_policy = '99PERCENTILE'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.UnifiedCompactionStrategy'}
    AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99PERCENTILE';

4. Create a new Family column with CQL code under the shell with an artificial primary key. Choose a partitioning key as well as a clustering key. The table in question must make sense with at least 3 columns;

CREATE TABLE dbs.newtable (
    id UUID,
    last_name text,
    family text,
    address text,
    PRIMARY KEY (id, last_name)
) WITH CLUSTERING ORDER BY (last_name ASC);

5. Insert two records to test the table.

INSERT INTO dbs.newtable (id, last_name, family, address) VALUES (uuid(), 'Doe', 'Smith', '123 Main Street');
INSERT INTO dbs.newtable (id, last_name, family, address) VALUES (uuid(), 'Doe', 'Jones', '456 Main Street');

6. Create a request which will respond with an error mentioning the use of “Allow filetting”.

SELECT * FROM dbs.netflix_userbase WHERE age = 30;

7. Add an index on each table and run a query that leverages the index usage.

CREATE INDEX ON dbs.netflix_userbase(age);
SELECT * FROM dbs.netflix_userbase WHERE age = 33;
CREATE INDEX ON dbs.customers(gender);
SELECT * FROM dbs.customers WHERE gender = 'Female';


8. Create a query with a filter with a condition on the null value of a column.

SELECT * FROM dbs.netflix_userbase WHERE device = 'NULL' ALLOW FILTERING;

9. Modify one of the tables by adding a Map type field.

ALTER TABLE dbs.netflix_userbase ADD preferences map<text, text>;
Run check: DESCRIBE dbs.netflix_userbase;

10. Insert a new record with values for the Map column.

INSERT INTO dbs.netflix_userbase (user_id, preferences) VALUES (12345, {'genre': 'Comedy', 'language': 'English'});
Run check: SELECT * FROM dbs.netflix_userbase WHERE user_id = 12345;

11. Return the number of records checking a condition on columns.

SELECT COUNT(*) FROM dbs.netflix_userbase WHERE monthly_revenue > 10 ALLOW FILTERING;

12. Change the value of a column for multiple records checking a condition.

By choosing a partition key, we can update multiple records at once. For example, if we want to update the monthly_revenue for all users in the United States, we can do so by using the country column as the partition key.

Set award as the partition key: ALTER TABLE dbs.netflix_userbase ADD award text;
Change partition key: ALTER TABLE dbs.netflix_userbase WITH PARTITION KEY (country, monthly_revenue);
Update the award for all users in the United States and > 10 monthly_revenue: UPDATE dbs.netflix_userbase SET award = 'yes' WHERE country = 'United States' AND monthly_revenue > 10;
Run check: SELECT * FROM dbs.netflix_userbase WHERE country = 'United States' AND monthly_revenue > 10;

13. Delete a record from a table

DELETE FROM dbs.netflix_userbase WHERE user_id = 12345;
Run check: SELECT * FROM dbs.netflix_userbase WHERE user_id = 12345;